### Data Ingestion workflow into Feast from Delta Lake table parquet files
<table>
  <tr><td>
    <img src="./images/delta_to_feast.png"
         alt="Workflow Data Ingestion from Delta Lake to Feast"  width="700">
  </td></tr>
</table>

In [85]:
import pandas as pd
from datetime import datetime
from utils import Utils
from feast import FeatureStore
from pprint import pprint

In [86]:
# entity_df generally comes from upstream systems
entity_df = pd.DataFrame.from_dict({
    "year_month_day": ['2014-01-01', '2014-01-05'],
    "event_timestamp": [
        datetime(2014, 1, 1, 8, 0, 0),
        datetime(2014, 1, 5, 8, 0, 0)
    ]
})

entity_df.head()

,year_month_day,event_timestamp
0,2014-01-01,2014-01-01 08:00:00
1,2014-01-05,2014-01-05 08:00:00


#### Get historical features from the Online Feature Store

In [87]:
feature_refs = [
        'weather_forecast_features:temperature_00',
        'weather_forecast_features:wind_direction_00',
        'weather_forecast_features:wind_speed_00',
        'weather_forecast_features:temperature_08',
        'weather_forecast_features:wind_direction_08',
        'weather_forecast_features:wind_speed_08',
        'weather_forecast_features:temperature_16',
        'weather_forecast_features:wind_direction_16',
        'weather_forecast_features:wind_speed_16',
        'weather_forecast_features:power',
        
    ]

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Get historical data for two entities

In [88]:
store = FeatureStore(repo_path=".") 
store.get_online_features
training_df = store.get_historical_features(
    entity_df=entity_df, 
    feature_refs=feature_refs
).to_df()

training_df

,event_timestamp,year_month_day,weather_forecast_features__temperature_00,weather_forecast_features__wind_direction_00,weather_forecast_features__wind_speed_00,weather_forecast_features__temperature_08,weather_forecast_features__wind_direction_08,weather_forecast_features__wind_speed_08,weather_forecast_features__temperature_16,weather_forecast_features__wind_direction_16,weather_forecast_features__wind_speed_16,weather_forecast_features__power
0,2014-01-01 08:00:00+00:00,2014-01-01,4.702022,106.74259,4.743292,7.189482,100.41638,6.593832,8.172301,99.2880,5.967206,1959.3535
1,2014-01-05 08:00:00+00:00,2014-01-05,0.830547,265.39440,4.263086,2.480239,104.79496,3.042063,4.227131,263.4169,3.899182,880.6115


### Get Online features for two entities

In [89]:
entity_rows = [{'year_month_day': "2014-01-01"}, {'year_month_day': '2014-01-05'}]

/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [90]:
online_response = store.get_online_features(feature_refs, entity_rows)

In [91]:
pprint(online_response.to_dict())

{'weather_forecast_features__power': [1959.3535, 880.6115],
 'weather_forecast_features__temperature_00': [4.702021725972501,
                                               0.8305474599202451],
 'weather_forecast_features__temperature_08': [7.189482116699223,
                                               2.480239041646324],
 'weather_forecast_features__temperature_16': [8.172300847371405,
                                               4.227130699157724],
 'weather_forecast_features__wind_direction_00': [106.74258999999999, 265.3944],
 'weather_forecast_features__wind_direction_08': [100.41638, 104.79496],
 'weather_forecast_features__wind_direction_16': [99.288, 263.4169],
 'weather_forecast_features__wind_speed_00': [4.743291999999999,
                                              4.263086299999999],
 'weather_forecast_features__wind_speed_08': [6.5938324999999995, 3.0420635],
 'weather_forecast_features__wind_speed_16': [5.967206, 3.899182],
 'year_month_day': ['2014-01-01', '2014-0